In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import time

In [3]:
from tensorflow.keras.models import load_model

In [4]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)  

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, h):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, h)
        self.fc2 = nn.Linear(h, h)
        self.fc3 = nn.Linear(h, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        return self.fc3(x)

net=Actor(6,1,time.time(),64)

In [5]:
dnn_model = load_model('dnn_model')

In [6]:
weights = dnn_model.get_weights()
weights

[array([ 1.497406 , 45.812748 ,  3.3521402,  3.5544746,  1.0940305,
         1.2861433], dtype=float32),
 array([0.24999326, 7.7099004 , 1.1658807 , 0.4999508 , 0.01168746,
        0.01274876], dtype=float32),
 1542,
 array([[ 1.31701976e-01, -6.11157537e-01, -5.88394344e-01,
          4.02225554e-01, -4.36646134e-01, -6.33917332e-01,
          4.85268682e-01, -5.73464334e-01, -5.02074063e-01,
         -1.93664446e-01,  5.05642116e-01,  7.64954805e-01,
          3.55253726e-01, -5.31777143e-01,  4.10381675e-01,
         -1.40102610e-01, -7.86653996e-01, -5.54282069e-01,
         -4.86459672e-01, -6.65316880e-01, -2.15517476e-01,
         -7.14815438e-01, -5.24070919e-01,  5.00249565e-01,
         -5.29301882e-01, -2.86954790e-01, -2.74895340e-01,
          6.70924842e-01,  3.85181993e-01,  6.16267323e-01,
          5.62021434e-01,  6.83333755e-01,  7.56498873e-01,
         -1.08270392e-01, -5.57310998e-01,  7.33952224e-01,
          5.84787667e-01,  5.32776654e-01,  5.25975406e-01,
   

In [7]:
net.fc1.weight.data=torch.from_numpy(np.transpose(weights[3]))
net.fc1.bias.data=torch.from_numpy(weights[4])
net.fc2.weight.data=torch.from_numpy(np.transpose(np.array(weights[5])))
net.fc2.bias.data=torch.from_numpy(weights[6])
net.fc3.weight.data=torch.from_numpy(np.transpose(weights[7]))
net.fc3.bias.data=torch.from_numpy(weights[8])

In [8]:
venue_lookup = pd.read_csv("venue.csv")
batsmen_lookup = pd.read_csv("batsmen.csv")
bowlers_lookup = pd.read_csv("bowlers.csv")

venue_lookup = venue_lookup.set_index("venue")["venue_mean_runs"].to_dict()
batsmen_lookup = batsmen_lookup.set_index("batsmen")["batsmen_mean_runs"].to_dict()
bowlers_lookup = bowlers_lookup.set_index("bowlers")["bowlers_mean_runs"].to_dict()

In [9]:
def predictor(testInput):
    prediction = 46.5
    
    batsman_mean = sum(batsmen_lookup.values())/len(batsmen_lookup)
    bowler_mean = sum(bowlers_lookup.values())/len(bowlers_lookup)
    
    df = pd.read_csv(testInput)
    
    df["venue_mean"] = 0
    df["count_batsmen"] = 0
    df["count_bowlers"] = 0
    df["batsmen_mean"] = 0
    df["bowlers_mean"] = 0
    
    batsmen = df["batsmen"][0].strip().split(',')
    bowlers = df["bowlers"][0].strip().split(',')
    venue = df["venue"][0]
    
    df["venue_mean"] = venue_lookup[venue]
    df["count_batsmen"] = len(batsmen)
    df["count_bowlers"] = len(bowlers)
    df["batsmen_mean"] = sum([batsmen_lookup.get(name, batsman_mean) for name in batsmen])/len(batsmen)
    df["bowlers_mean"] = sum([bowlers_lookup.get(name, bowler_mean) for name in bowlers])/len(bowlers)
    
    df = df.drop(columns = ["venue", "batsmen", "bowlers", "batting_team", "bowling_team"])
    
    df = df.to_numpy()
    
    df = np.clip((df - weights[0])/(np.sqrt(weights[1]) + 0.001), -1, 1)
    
    df = torch.from_numpy(df.astype(np.float32))
    
    prediction = net(df)
    
    return prediction[0, 0]

In [10]:
predictor("trial/18_inn2.csv")

tensor(42.7177, grad_fn=<SelectBackward>)

In [11]:
torch.save(net.state_dict(), "pytorch_model.pt")

In [12]:
dnn_model = Actor(6,1,time.time(),64)
dnn_model.load_state_dict(torch.load("pytorch_model.pt"))

<All keys matched successfully>

In [13]:
def predictor2(testInput):
    prediction = 46.5
    
    batsman_mean = sum(batsmen_lookup.values())/len(batsmen_lookup)
    bowler_mean = sum(bowlers_lookup.values())/len(bowlers_lookup)
    
    df = pd.read_csv(testInput)
    
    df["venue_mean"] = 0
    df["count_batsmen"] = 0
    df["count_bowlers"] = 0
    df["batsmen_mean"] = 0
    df["bowlers_mean"] = 0
    
    batsmen = df["batsmen"][0].strip().split(',')
    bowlers = df["bowlers"][0].strip().split(',')
    venue = df["venue"][0]
    
    df["venue_mean"] = venue_lookup[venue]
    df["count_batsmen"] = len(batsmen)
    df["count_bowlers"] = len(bowlers)
    df["batsmen_mean"] = sum([batsmen_lookup.get(name, batsman_mean) for name in batsmen])/len(batsmen)
    df["bowlers_mean"] = sum([bowlers_lookup.get(name, bowler_mean) for name in bowlers])/len(bowlers)
    
    df = df.drop(columns = ["venue", "batsmen", "bowlers", "batting_team", "bowling_team"])
    
    df = df.to_numpy()
    
    df = (df - weights[0])/(np.sqrt(weights[1]) + 0.001)
    
    df = torch.from_numpy(df.astype(np.float32))
    
    prediction = dnn_model(df)
    
    return prediction[0, 0]

In [15]:
int(predictor2("trial/18_inn1.csv"))

46